In [ ]:
from typing import List
import qrcode
import os
from PIL import Image
import requests
from bs4 import BeautifulSoup

def get_artwork(code: str):
    img_filename = f"content/{code}_img.jpg"
    qr_filename = f"content/{code}_qr.jpg"
    url = f"https://www.platform-a.art/work@{code}"

    # Download image
    if os.path.exists(img_filename) and os.path.exists(qr_filename):
        print(f'{code}: already downloaded')
        return
    
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    element = soup.find(attrs={"property": "og:image"})

    if not element:
        raise Exception("image element not found")
    
    img_url = element.attrs['content']
    response = requests.get(img_url)
    if response.status_code == 200:
        # Open a file in binary write mode and save the image
        with open(img_filename, 'wb') as file:
            file.write(response.content)
    else:
        raise Exception("couldn't download image")
        
    # Generate the QR code
    qr = qrcode.QRCode(
       version=1,
       error_correction=qrcode.constants.ERROR_CORRECT_L,
       box_size=10,
       border=4,
    )
    qr.add_data(url)
    qr.make(fit=True)

    img = qr.make_image(fill='black', back_color='white')
    img = img.convert("RGB")  # Convert to RGB mode
    img.save(qr_filename)

def get_artworks(codes: List[str]):
    for code in codes:
        try:
            get_artwork(code)
            print(f'{code} done.')
        except Exception as e:
            print(f'{code} failed: {e}')


In [ ]:
template=r"""\documentclass{article}
\RequirePackage{graphicx}
\RequirePackage{geometry}
\RequirePackage{etoolbox}
\RequirePackage[most]{tcolorbox}
\RequirePackage{setspace}
\RequirePackage{fontspec}
\RequirePackage{xeCJK} % Load the package
\setmainfont{Noto Sans CJK HK}
\setCJKmainfont{Noto Sans CJK HK}


\geometry{
	a4paper,
	landscape,
	margin=0in
}

\newcommand{\stickerwidth}{42.3mm}
\newcommand{\stickerheight}{105mm}
\newcommand{\imagesize}{40mm}
\newcommand{\qrsize}{20mm}


\newcommand{\sticker}[7] {
	% #1: code
	% #2: title
	% #3: artist
	% #4: medium
	% #5: width
	% #6: height
	% #7: price	
	
	\begin{tcolorbox}[
		width=\stickerwidth,
		height=\stickerheight,
		colframe=white,
		colback=white,
		sharp corners,
		boxsep=0pt,
		left=0pt, right=0pt, bottom=0pt, top=0pt,
		boxrule=0pt,
		valign=top]

		\vspace{2mm}
		\begin{minipage}[c][\imagesize]{\textwidth}
			\centering
	
			\includegraphics[width=\imagesize,height=\imagesize,keepaspectratio]{content/#1_img.jpg} % Image
		\end{minipage}
	
		\begin{minipage}[c][4cm]{\textwidth}
			\centering
			\vspace{4mm}
			#2
			
			\vspace{9mm}

			#3

			\vspace{2mm}
			
			#4

			\vspace{2mm}
			
			#7 NTD
		\end{minipage}

		\begin{minipage}[b]{\textwidth}
			\centering
			\includegraphics[width=\qrsize]{content/#1_qr.jpg} % Qr code
		\end{minipage}

		\vspace{1cm}		


	\end{tcolorbox}
}

\begin{document}
	\pagestyle{empty} % Removes page numbers

\begin{flushleft}
	\begin{tcbitemize}[raster columns=7,
					   raster equal height,
					   raster row skip=0mm,
					   raster column skip=0mm,
					   colback=white,
					   colframe=white,
					   size=tight,
					   boxrule=0pt,
					   boxsep=0pt,
					   halign=center,
					   valign=center]					   
###WORKS###
	\end{tcbitemize}
\end{flushleft}

\end{document}"""

In [ ]:
import csv
with open('works.csv', 'r') as f:
    reader = csv.reader(f)
    works = [row for row in csv.reader(f)][1:]
codes = [w[2] for w in works]

get_artworks(codes)
# columns in csv:
#  0 作品名稱
#  1 藝術家
#  2 code
#  3 媒材
#  4 作品尺寸(高)
#  5 作品尺寸(寬)
#  6 作品售價NTD
# columns in template:
#1: code
#2: title
#3: artist
#4: medium
#5: width
#6: height
#7: price
tex_lines = '\n'.join([f'\\tcbitem \\sticker{{{w[2]}}}{{{w[0].replace("#", "\\#")}}}{{{w[1]}}}{{{w[3]}}}{{{w[4]}}}{{{w[5]}}}{{{w[6]}}}' for w in works])
tex = (template.replace("###WORKS###", tex_lines))

with open('GENERATED.tex', 'w') as file:
    file.write(tex)